## California Dataset Pre-Processing: Retrieve Zipcodes

Retrieve an approximate street address from Google Maps for each latitude and longitude.  Extract zip code from address.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

import time
import re 
import os

from dotenv import load_dotenv
from geopy.geocoders import GoogleV3

In [ ]:
load_dotenv()

gc = GoogleV3(api_key=os.environ.get("GOOGLE_API_KEY"))

temp = fetch_california_housing()
df = pd.DataFrame(temp.data, columns=temp.feature_names)
df["housing_price"] = pd.DataFrame(temp.target, columns=temp.target_names)
df.dropna(axis=1, inplace=True)

In [ ]:
pattern = re.compile("CA 9[0-9]{4}")

def get_zipcode(lat, long):
    x = gc.reverse((lat, long))
    print(x.address)
    zip = re.findall(pattern, x.address)
    time.sleep(1)
    if len(zip) > 0:
        return zip[0].split(" ")[1]
    else:
        return None 

for i in np.arange(0,20000,100):
    print(i)
    df.loc[(df.index>i) & (df.index<=(i+100)) & (df.zipcode.isna()), "zipcode"] = df.loc[(df.index>i) & (df.index<=(i+100)) & (df.zipcode.isna()),["Latitude","Longitude"]].apply(lambda x: get_zipcode(x.iloc[0], x.iloc[1]), axis=1)
    time.sleep(1)
    df.to_csv("housing.csv", index=True)